<a href="https://colab.research.google.com/github/AirbornBird88/hmill-exper/blob/main/Arxiv_classification_with_bert_multilabel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Klasifikace článků na ArXivu do kategorií pomocí BERT modelu (fine tuning přístup)

In [ ]:
pip install transformers

## Přečíst ArXiv dataset a jeho předzpracování do vhodné podoby

In [ ]:
!pip install kaggle

In [ ]:
# Upload Kaggle API Key
from google.colab import files
files.upload()  # Upload the kaggle.json file

Saving kaggle_mill.json to kaggle_mill.json


{'kaggle_mill.json': b'{"username":"airbornbird88","key":"523660c5fbf3cccd2135247a1a03565c"}'}

In [ ]:
# Set up Kaggle configuration
!mkdir -p ~/.kaggle
!mv kaggle_mill.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle_mill.json

In [ ]:
# Download the arXiv dataset
!kaggle datasets download -d Cornell-University/arxiv

Dataset URL: https://www.kaggle.com/datasets/Cornell-University/arxiv
License(s): CC0-1.0
100% 1.29G/1.29G [00:33<00:00, 41.4MB/s]
100% 1.29G/1.29G [00:33<00:00, 41.2MB/s]


In [ ]:
# Extract the dataset
!unzip arxiv.zip

Archive:  arxiv.zip
  inflating: arxiv-metadata-oai-snapshot.json  


In [ ]:
ls /content

arxiv-metadata-oai-snapshot.json  arxiv.zip  sample_data/


In [ ]:
import json

# File path
data_file = "/content/arxiv-metadata-oai-snapshot.json"

# List to store parsed JSON objects
samples = []

# Maximum number of samples to parse
max_samples = 5000

# Open the file and read line by line
with open(data_file, 'r') as file:
    for line in file:
        # Parse each line as a JSON object and append it to the samples list
        samples.append(json.loads(line))

        # Check if we've reached the maximum number of samples
        if len(samples) >= max_samples:
            break

# samples now contains the parsed JSON objects


In [ ]:
# Assuming 'samples' contains the parsed JSON objects
# Print example of multiple JSON objects in samples
for i in range(5):  # Adjust the range as needed
    print(json.dumps(samples[i], indent=2))


In [ ]:
# Convert JSON data to DataFrame
df = pd.json_normalize(samples)

In [ ]:
df.head()

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"
2,0704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]"
3,0704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Callan, David, ]]"
4,0704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]"


In [ ]:
# Function to concatenate all fields into a single text column
def concatenate_fields(row):
    text = []
    for key, value in row.items():
        if key not in ['id', 'categories'] and value is not None:
            text.append(f"{key}: {value}")
    return ". ".join(text)

In [ ]:
# Apply the function to create the Text column
df['Text'] = df.apply(concatenate_fields, axis=1)

In [ ]:
# Keep only the necessary columns
df = df[['id', 'Text', 'categories']]
df.columns = ['ArticleId', 'Text', 'Categories']

In [ ]:
df.head()

,ArticleId,Text,Categories
0,0704.0001,submitter: Pavel Nadolsky. authors: C. Bal\'az...,hep-ph
1,0704.0002,submitter: Louis Theran. authors: Ileana Strei...,math.CO cs.CG
2,0704.0003,submitter: Hongjun Pan. authors: Hongjun Pan. ...,physics.gen-ph
3,0704.0004,submitter: David Callan. authors: David Callan...,math.CO
4,0704.0005,submitter: Alberto Torchinsky. authors: Wael A...,math.CA math.FA


In [ ]:
print(f"Total records: {len(df)}")
categories = df['Categories'].unique()
print(f"Categories: {categories.tolist()}")
df.head()

Total records: 5000
Categories: ['hep-ph', 'math.CO cs.CG', 'physics.gen-ph', 'math.CO', 'math.CA math.FA', 'cond-mat.mes-hall', 'gr-qc', 'cond-mat.mtrl-sci', 'astro-ph', 'math.NT math.AG', 'math.NT', 'math.CA math.AT', 'hep-th', 'math.PR math.AG', 'hep-ex', 'nlin.PS physics.chem-ph q-bio.MN', 'math.NA', 'nlin.PS', 'cond-mat.str-el cond-mat.stat-mech', 'math.RA', 'math.CA math.PR', 'cond-mat.str-el', 'physics.optics physics.comp-ph', 'q-bio.PE q-bio.CB quant-ph', 'q-bio.QM q-bio.MN', 'hep-ph hep-lat nucl-th', 'math.OA math.FA', 'math.QA math-ph math.MP', 'physics.gen-ph quant-ph', 'cond-mat.stat-mech cond-mat.mtrl-sci', 'astro-ph nlin.CD physics.plasm-ph physics.space-ph', 'nlin.PS nlin.SI', 'quant-ph cs.IT math.IT', 'cs.NE cs.AI', 'gr-qc astro-ph', 'physics.ed-ph quant-ph', 'math.DG gr-qc', 'cond-mat.soft cond-mat.mtrl-sci', 'physics.pop-ph', 'nucl-th', 'math.FA', 'cs.DS', 'math.AG math.CO', 'math.NT math.CV', 'math.DS', 'physics.soc-ph', 'math-ph math.MP', 'math.AG', 'hep-ph hep-ex n

,ArticleId,Text,Categories
0,0704.0001,submitter: Pavel Nadolsky. authors: C. Bal\'az...,hep-ph
1,0704.0002,submitter: Louis Theran. authors: Ileana Strei...,math.CO cs.CG
2,0704.0003,submitter: Hongjun Pan. authors: Hongjun Pan. ...,physics.gen-ph
3,0704.0004,submitter: David Callan. authors: David Callan...,math.CO
4,0704.0005,submitter: Alberto Torchinsky. authors: Wael A...,math.CA math.FA


In [ ]:
# Splitting categories and creating new rows for each label
new_df = df.assign(Categories=df['Categories'].str.split()).explode('Categories')

# One-hot encoding
one_hot_encoded = pd.get_dummies(new_df, columns=['Categories'])

# Group by ArticleId and sum to get counts for each label
grouped_df = one_hot_encoded.groupby('ArticleId').sum()

# If you want to retain the text column, you can merge it back to the grouped DataFrame
final_df = grouped_df.merge(df[['ArticleId', 'Text']], on='ArticleId')

In [ ]:
print(final_df)

      ArticleId                                             Text_x  \
0     0704.0001  submitter: Pavel Nadolsky. authors: C. Bal\'az...   
1     0704.0002  submitter: Louis Theran. authors: Ileana Strei...   
2     0704.0003  submitter: Hongjun Pan. authors: Hongjun Pan. ...   
3     0704.0004  submitter: David Callan. authors: David Callan...   
4     0704.0005  submitter: Alberto Torchinsky. authors: Wael A...   
...         ...                                                ...   
4995  0705.0993  submitter: Heather Knutson. authors: Heather A...   
4996  0705.0994  submitter: Andreas Karch. authors: Dongsu Bak,...   
4997  0705.0995  submitter: Zhongyuan Zhou. authors: Zhongyuan ...   
4998  0705.0996  submitter: Anzhong Wang. authors: Yungui Gong ...   
4999  0705.0997  submitter: Daniel V. Shantsev. authors: Age An...   

      Categories_astro-ph  Categories_astro-ph.EP  Categories_astro-ph.HE  \
0                       0                       0                       0   
1    

In [ ]:
df = final_df

In [ ]:
df.head()

,ArticleId,Text_x,Categories_astro-ph,Categories_astro-ph.EP,Categories_astro-ph.HE,Categories_cond-mat.dis-nn,Categories_cond-mat.mes-hall,Categories_cond-mat.mtrl-sci,Categories_cond-mat.other,Categories_cond-mat.quant-gas,...,Categories_q-fin.PR,Categories_q-fin.RM,Categories_q-fin.ST,Categories_quant-ph,Categories_stat.AP,Categories_stat.CO,Categories_stat.ME,Categories_stat.ML,Categories_stat.TH,Text_y
0,0704.0001,submitter: Pavel Nadolsky. authors: C. Bal\'az...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,submitter: Pavel Nadolsky. authors: C. Bal\'az...
1,0704.0002,submitter: Louis Theran. authors: Ileana Strei...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,submitter: Louis Theran. authors: Ileana Strei...
2,0704.0003,submitter: Hongjun Pan. authors: Hongjun Pan. ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,submitter: Hongjun Pan. authors: Hongjun Pan. ...
3,0704.0004,submitter: David Callan. authors: David Callan...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,submitter: David Callan. authors: David Callan...
4,0704.0005,submitter: Alberto Torchinsky. authors: Wael A...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,submitter: Alberto Torchinsky. authors: Wael A...


In [ ]:
# Group by 'Categories' and get the size of each group
category_counts = df.groupby(['Categories']).size().reset_index(name='Count')

# Sort the categories by count for better readability
category_counts = category_counts.sort_values(by='Count', ascending=False)

# Display the resulting DataFrame
print(category_counts)

                         Categories  Count
0                          astro-ph    915
283                          hep-ph    231
910                        quant-ph    205
305                          hep-th    193
50                cond-mat.mtrl-sci    111
..                              ...    ...
441                 math.CA math.DS      1
69   cond-mat.other hep-th quant-ph      1
443                 math.CA math.OA      1
444                 math.CA math.PR      1
942  stat.ME physics.soc-ph stat.AP      1

[943 rows x 2 columns]


In [ ]:
# Total count of articles
total_articles = len(df)

# Sum of counts for individual categories
category_counts_sum = category_counts['Count'].sum()

# Compare total count of articles with the sum of counts for individual categories
print("Total count of articles:", total_articles)
print("Sum of counts for individual categories:", category_counts_sum)

Total count of articles: 5000
Sum of counts for individual categories: 7573


## Load Bert model a vyzkoušet jeho Tokenizer

In [ ]:
from transformers import BertTokenizer
from transformers.tokenization_utils_base import BatchEncoding

transformer_model_id = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(transformer_model_id)

example_text = 'A fully differential calculation in perturbative quantum chromodynamics is presented for the production of massive photon pairs at hadron colliders. All next-to-leading order perturbative contributions from quark-antiquark, gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as all-orders resummation of initial-state gluon radiation valid at next-to-next-to-leading logarithmic accuracy. The region of phase space is specified in which the calculation is most reliable. Good agreement is demonstrated with data from the Fermilab Tevatron, and predictions are made for more detailed tests with CDF and DO data. Predictions are shown for distributions of diphoton pairs produced at the energy of the Large Hadron Collider (LHC). Distributions of the diphoton pairs from the decay of a Higgs boson are contrasted with those produced from QCD processes at the LHC, showing that enhanced sensitivity to the signal can be obtained with judicious selection of events. '
tokens = tokenizer.tokenize(text=example_text)
print(f"Tokens: {tokens}")

bert_input = tokenizer(example_text, padding='max_length', max_length = 50,
                       truncation=True, return_tensors="pt")
print(type(bert_input))
print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

Tokens: ['A', 'fully', 'differential', 'calculation', 'in', 'per', '##tur', '##bati', '##ve', 'quantum', 'ch', '##romo', '##dyn', '##ami', '##cs', 'is', 'presented', 'for', 'the', 'production', 'of', 'massive', 'photo', '##n', 'pairs', 'at', 'had', '##ron', 'coll', '##ider', '##s', '.', 'All', 'next', '-', 'to', '-', 'leading', 'order', 'per', '##tur', '##bati', '##ve', 'contributions', 'from', 'qua', '##rk', '-', 'anti', '##qua', '##rk', ',', 'g', '##lu', '##on', '-', '(', 'anti', ')', 'qua', '##rk', ',', 'and', 'g', '##lu', '##on', '-', 'g', '##lu', '##on', 'sub', '##pro', '##cesse', '##s', 'are', 'included', ',', 'as', 'well', 'as', 'all', '-', 'orders', 'res', '##umm', '##ation', 'of', 'initial', '-', 'state', 'g', '##lu', '##on', 'radiation', 'valid', 'at', 'next', '-', 'to', '-', 'next', '-', 'to', '-', 'leading', 'log', '##ari', '##th', '##mic', 'accuracy', '.', 'The', 'region', 'of', 'phase', 'space', 'is', 'specified', 'in', 'which', 'the', 'calculation', 'is', 'most', 'reliab

In [ ]:
tokens = tokenizer.tokenize(example_text)
print("Number of tokens:", len(tokens))

Number of tokens: 239


In [ ]:
token_ids = bert_input['input_ids'][0]
print(f"Tokens again: {tokenizer.convert_ids_to_tokens(token_ids)}")
print(f"Words: {tokenizer.decode(token_ids)}")

Tokens again: ['[CLS]', 'A', 'fully', 'differential', 'calculation', 'in', 'per', '##tur', '##bati', '##ve', 'quantum', 'ch', '##romo', '##dyn', '##ami', '##cs', 'is', 'presented', 'for', 'the', 'production', 'of', 'massive', 'photo', '##n', 'pairs', 'at', 'had', '##ron', 'coll', '##ider', '##s', '.', 'All', 'next', '-', 'to', '-', 'leading', 'order', 'per', '##tur', '##bati', '##ve', 'contributions', 'from', 'qua', '##rk', '-', '[SEP]']
Words: [CLS] A fully differential calculation in perturbative quantum chromodynamics is presented for the production of massive photon pairs at hadron colliders. All next - to - leading order perturbative contributions from quark - [SEP]


## Připravit ArXiv dataset pro trénování modelu

In [ ]:
# Get unique categories from your DataFrame
unique_categories = category_counts['Categories'].unique()

# Assuming num_categories is the number of categories in arXiv data
num_categories = len(unique_categories)
num_categories

135

In [ ]:
import numpy as np
import torch
from pandas import DataFrame

class ARXIVDataset(torch.utils.data.Dataset):
    def __init__(self, data_frame: DataFrame):
        self.labels = [np.array(label, dtype=np.float32) for label in data_frame.iloc[:, 2:].values]
        self.texts = [text for text in data_frame['Text']]

    def classes(self) -> list[int]:
        return self.labels

    def __len__(self) -> int:
        return len(self.labels)

    def get_batch_labels(self, idx: int) -> np.array:
        return self.labels[idx]

    def get_batch_texts(self, idx: int) -> str:
        return self.texts[idx]

    def __getitem__(self, idx: int) -> tuple:
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = tokenizer(text, padding='max_length', max_length=640, truncation=True, return_tensors="pt")
        input_ids = encoding['input_ids'].squeeze(0)  # Remove batch dimension
        attention_mask = encoding['attention_mask'].squeeze(0)  # Remove batch dimension
        return input_ids, attention_mask, label

In [ ]:
import numpy as np
import torch
from pandas import DataFrame

class ARXIVDataset(torch.utils.data.Dataset):
    def __init__(self, data_frame: DataFrame):
        # Assuming the dummy categories start from the third column
        self.labels = [label.astype(np.float32) for label in data_frame.iloc[:, 2:].values]
        self.texts = data_frame['Text_x'].tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = torch.tensor(self.labels[idx], dtype=torch.float32)  # Convert to tensor
        text = self.texts[idx]
        encoding = tokenizer(text, padding='max_length', max_length=640, truncation=True, return_tensors="pt")
        input_ids = encoding['input_ids'].squeeze(0)  # Remove batch dimension
        attention_mask = encoding['attention_mask'].squeeze(0)  # Remove batch dimension
        return input_ids, attention_mask, label


In [ ]:
np.random.seed(42)
# randomize all examples and split them into 80th percentile and 90th percentile
# per training, validation and test set:
df_train, df_val, df_test = np.split(
    df.sample(frac=1, random_state=42),
    [int(.8 * len(df)), int(.9 * len(df))]
    )

print(f"{len(df)} divided to: {len(df_train)} train, {len(df_val)} validation, {len(df_test)} test")

5000 divided to: 4000 train, 500 validation, 500 test


# Architektura sítě založené na Bert transformeru

In [ ]:
from torch import nn
import torch.nn.functional as F
from transformers import BertModel

class BertMultiLabelClassifier(nn.Module):
    def __init__(self, num_categories, dropout=0.5):
        super(BertMultiLabelClassifier, self).__init__()

        self.bert = BertModel.from_pretrained(transformer_model_id)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, num_categories)

    def forward(self, input_ids, mask):
        contextualized_token_embeddings, pooled_output = self.bert(input_ids=input_ids, attention_mask=mask, return_dict=False)
        dropout_output = self.dropout(pooled_output)
        logits = self.linear(dropout_output)
        probabilities = torch.sigmoid(logits)  # Apply sigmoid activation for multi-label classification
        return probabilities

# Model natrénovat/dotrénovat na klasifikační úlohu pomocí trénovacích a validačních dat

---



In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
if use_cuda:
    print("Using CUDA")

In [ ]:
# Display column names
print(df.columns)

# Display first few rows of the DataFrame
print(df.head())


Index(['ArticleId', 'Text_x', 'Categories_astro-ph', 'Categories_astro-ph.EP',
       'Categories_astro-ph.HE', 'Categories_cond-mat.dis-nn',
       'Categories_cond-mat.mes-hall', 'Categories_cond-mat.mtrl-sci',
       'Categories_cond-mat.other', 'Categories_cond-mat.quant-gas',
       ...
       'Categories_q-fin.PR', 'Categories_q-fin.RM', 'Categories_q-fin.ST',
       'Categories_quant-ph', 'Categories_stat.AP', 'Categories_stat.CO',
       'Categories_stat.ME', 'Categories_stat.ML', 'Categories_stat.TH',
       'Text_y'],
      dtype='object', length=138)
   ArticleId                                             Text_x  \
0  0704.0001  submitter: Pavel Nadolsky. authors: C. Bal\'az...   
1  0704.0002  submitter: Louis Theran. authors: Ileana Strei...   
2  0704.0003  submitter: Hongjun Pan. authors: Hongjun Pan. ...   
3  0704.0004  submitter: David Callan. authors: David Callan...   
4  0704.0005  submitter: Alberto Torchinsky. authors: Wael A...   

   Categories_astro-ph  Categ

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):
    train_dataset, val_dataset = ARXIVDataset(train_data), ARXIVDataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2)

    criterion = nn.BCELoss()  # Use binary cross-entropy loss for multi-label classification
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if use_cuda:
        model = model.cuda()

    for epoch_num in range(epochs):
        total_loss_train = 0
        total_loss_val = 0
        total_acc_train = 0
        total_acc_val = 0

        model.train()  # Set the model to training mode
        for train_input_ids, train_attention_mask, train_label in tqdm(train_dataloader):
            train_input_ids = train_input_ids.to(device)
            train_attention_mask = train_attention_mask.to(device)
            train_label = train_label.float().to(device)  # Convert labels to float for BCELoss

            optimizer.zero_grad()

            output = model(train_input_ids, train_attention_mask)

            batch_loss = criterion(output, train_label)
            total_loss_train += batch_loss.item()

            batch_acc = ((output > 0.5).int() == train_label.int()).sum().item()  # Calculate accuracy
            total_acc_train += batch_acc

            batch_loss.backward()
            optimizer.step()

        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():
            for val_input_ids, val_attention_mask, val_label in val_dataloader:
                val_input_ids = val_input_ids.to(device)
                val_attention_mask = val_attention_mask.to(device)
                val_label = val_label.float().to(device)

                output = model(val_input_ids, val_attention_mask)

                batch_loss = criterion(output, val_label)
                total_loss_val += batch_loss.item()

                batch_acc = ((output > 0.5).int() == val_label.int()).sum().item()
                total_acc_val += batch_acc

        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset):.3f} | Train Accuracy: {total_acc_train / len(train_dataset):.3f} | '
            f'Val Loss: {total_loss_val / len(val_dataset):.3f} | Val Accuracy: {total_acc_val / len(val_dataset):.3f}')


EPOCHS = 5
model = BertMultiLabelClassifier(num_categories)  # Adjusted for multi-label classification
LR = 1e-6

train(model, df_train, df_val, LR, EPOCHS)


ValueError: could not convert string to float: 'submitter: Markus Sch\\"oller. authors: S. Hubrig, P. North, M. Schoeller. title: Evolution of Magnetic Fields in Stars Across the Upper Main Sequence:\n  II. Observed distribution of the magnetic field geometry. comments: 16 pages, 22 figures, 5 tables, accepted for publication in AN. doi: 10.1002/asna.200710773. abstract:   We re-discuss the evolutionary state of upper main sequence magnetic stars\nusing a sample of Ap and Bp stars with accurate Hipparcos parallaxes and\ndefinitely determined longitudinal magnetic fields. We confirm our previous\nresults obtained from the study of Ap and Bp stars with accurate measurements\nof the mean magnetic field modulus and mean quadratic magnetic fields that\nmagnetic stars of mass M < 3 M_sun are concentrated towards the centre of the\nmain-sequence band. In contrast, stars with masses M > 3 M_sun seem to be\nconcentrated closer to the ZAMS. The study of a few known members of nearby\nopen cluste

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

def train(model: BertClassifier, train_data: DataFrame, val_data: DataFrame, learning_rate: float, epochs: int):

    train, val = ARXIVDataset(train_data), ARXIVDataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):
            total_acc_train = 0
            total_loss_train = 0
            for train_input_ids, train_attention_mask, train_label in tqdm(train_dataloader):
                train_label = train_label.to(device)
                train_input_ids = train_input_ids.to(device)
                train_attention_mask = train_attention_mask.to(device)

                output = model(train_input_ids, train_attention_mask)

                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()


            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input_ids, val_attention_mask, val_label in val_dataloader:
                    val_label = val_label.to(device)
                    val_input_ids = val_input_ids.to(device)
                    val_attention_mask = val_attention_mask.to(device)

                    output = model(val_input_ids, val_attention_mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc


            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')

EPOCHS = 5
model = BertClassifier(num_categories)
LR = 1e-6

train(model, df_train, df_val, LR, EPOCHS)

100%|██████████| 2000/2000 [07:59<00:00,  4.17it/s]


Epochs: 1 | Train Loss:  3.108                 | Train Accuracy:  0.153                 | Val Loss:  2.902                 | Val Accuracy:  0.184


100%|██████████| 2000/2000 [07:58<00:00,  4.18it/s]


Epochs: 2 | Train Loss:  2.796                 | Train Accuracy:  0.184                 | Val Loss:  2.725                 | Val Accuracy:  0.184


100%|██████████| 2000/2000 [07:58<00:00,  4.18it/s]


Epochs: 3 | Train Loss:  2.624                 | Train Accuracy:  0.206                 | Val Loss:  2.600                 | Val Accuracy:  0.212


100%|██████████| 2000/2000 [07:58<00:00,  4.18it/s]


Epochs: 4 | Train Loss:  2.522                 | Train Accuracy:  0.238                 | Val Loss:  2.535                 | Val Accuracy:  0.224


100%|██████████| 2000/2000 [07:59<00:00,  4.17it/s]


Epochs: 5 | Train Loss:  2.449                 | Train Accuracy:  0.262                 | Val Loss:  2.476                 | Val Accuracy:  0.256


Vyzkoušet model na neviděných testovacích datech

In [ ]:
def evaluate(model: BertClassifier, test_data: DataFrame):
    test = ARXIVDataset(test_data)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():
        for test_batch in test_dataloader:
            input_ids, attention_mask, test_label = test_batch

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            test_label = test_label.to(device)

            output = model(input_ids, attention_mask)
            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

evaluate(model, df_test)

Test Accuracy:  0.260


In [ ]:
# File path
data_file = "/content/arxiv-metadata-oai-snapshot.json"

# Function to load a random sample from the JSON data
def load_random_sample(file_path):
    with open(file_path, 'r') as file:
        # Read the entire file into memory
        lines = file.readlines()
        # Choose a random line
        random_line = random.choice(lines)
        # Parse the JSON object from the chosen line
        random_sample = json.loads(random_line)
    return random_sample

# Function to concatenate all fields into a single text column
def concatenate_fields(row):
    text = []
    for key, value in row.items():
        if key not in ['id', 'categories'] and value is not None:
            text.append(f"{key}: {value}")
    return ". ".join(text)

# Load a random sample
random_sample = load_random_sample(data_file)

# Normalize the random sample to DataFrame
df = pd.json_normalize([random_sample])

# Apply the function to create the Text column
df['Text'] = df.apply(concatenate_fields, axis=1)

# Keep only the necessary columns
df = df[['id', 'Text', 'categories']]
df.columns = ['ArticleId', 'Text', 'Categories']

# Tokenize the text using the BERT tokenizer
bert_input = tokenizer(
    df['Text'].iloc[0],  # Assuming the text column is named 'Text'
    padding='max_length',
    max_length=512,  # Adjust the maximum length if needed
    truncation=True,
    return_tensors="pt"
)

# Pass the tokenized input to your model to obtain predictions
with torch.no_grad():
    model_output = model(
        bert_input['input_ids'].to(device),
        bert_input['attention_mask'].to(device)
    )

    # Obtain the index of the predicted category
    best_category_idx = model_output.argmax(dim=1).item()

    # Decode the predicted category index into its corresponding label
    best_category = list(category2index.keys())[best_category_idx]

    # Print predictions along with article ID, text, and categories
    print(f"Article ID: {df['ArticleId'].iloc[0]}")
    print(f"Text: {df['Text'].iloc[0]}")
    print(f"Categories: {df['Categories'].iloc[0]}")
    print(f"Predicted Category: {best_category}")


Article ID: math/0407057
Text: submitter: R. J. Williams. authors: F. P. Kelly and R. J. Williams. title: Fluid model for a network operating under a fair bandwidth-sharing
  policy. journal-ref: Annals of Applied Probability 2004, Vol. 14, No. 3, 1055-1083. doi: 10.1214/105051604000000224. report-no: IMS-AAP-AAP007. abstract:   We consider a model of Internet congestion control that represents the
randomly varying number of flows present in a network where bandwidth is shared
fairly between document transfers. We study critical fluid models obtained as
formal limits under law of large numbers scalings when the average load on at
least one resource is equal to its capacity. We establish convergence to
equilibria for fluid models and identify the invariant manifold.
  The form of the invariant manifold gives insight into the phenomenon of
entrainment whereby congestion at some resources may prevent other resources
from working at their full capacity.
. versions: [{'version': 'v1', 'crea

In [ ]:
# Define an example text
example_text = "The robot discovered a new species of plant on Mars during its exploration mission."

# Tokenize the text using the BERT tokenizer
bert_input = tokenizer(
    example_text,
    padding='max_length',
    max_length=512,  # Adjust the maximum length if needed
    truncation=True,
    return_tensors="pt"
)

# Pass the tokenized input to your model to obtain predictions
with torch.no_grad():
    model_output = model(
        bert_input['input_ids'].to(device),
        bert_input['attention_mask'].to(device)
    )

    # Obtain the index of the predicted category
    best_category_idx = model_output.argmax(dim=1).item()

    # Decode the predicted category index into its corresponding label
    best_category = list(category2index.keys())[best_category_idx]

    # Print predictions
    print(f"Predictions: {model_output}")
    print(f"Best class index: {best_category_idx}, which corresponds to: {best_category}")


Predictions: tensor([[5.9322e+00, 8.8942e-01, 6.5705e-01, 3.5424e-01, 1.4550e+00, 8.0532e-01,
         5.3074e-01, 7.7017e-01, 1.8792e-01, 8.4944e-01, 5.2396e-01, 1.1922e+00,
         0.0000e+00, 1.9343e-01, 3.0919e-01, 0.0000e+00, 8.2754e-01, 2.3908e-01,
         7.9553e-01, 0.0000e+00, 0.0000e+00, 1.4217e-02, 4.6964e-01, 0.0000e+00,
         4.7644e-01, 5.7863e-01, 3.9653e-01, 2.7557e-01, 6.8310e-02, 3.2013e-01,
         1.1790e-01, 1.2455e-01, 0.0000e+00, 0.0000e+00, 4.7703e-02, 3.3031e-02,
         4.2478e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 2.2362e-01, 5.2508e-01,
         0.0000e+00, 5.2389e-01, 5.9085e-01, 9.2852e-03, 0.0000e+00, 0.0000e+00,
         8.8889e-02, 5.8919e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 2.2762e-01,
         0.0000e+00, 0.0000e+00, 3.4966e-01, 0.0000e+00, 0.0000e+00, 2.5742e-01,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.7648e-01, 7.0035e-04, 0.0000e+00,
         7.9559e-01, 9.0129e-02, 1.9757e-01, 6.8420e-02, 0.0000e+00, 0.0000e+00,
         0.0000

In [ ]:
# Assuming you have already loaded your trained BERT model and assigned it to the variable 'model'
# Example:
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Count the total number of parameters
total_params = sum(p.numel() for p in model.parameters())

print(f"Total number of parameters in the model: {total_params}")

Total number of parameters in the model: 178578607
